In [42]:
# A Simple RAG Pipeline
import os
from dotenv import load_dotenv, dotenv_values
from utils import clean_text, print_chunks, reset_pinecone, embed_and_upsert

# Load All ENV Values
load_dotenv()
pinecone_namespace="simple-rag-1-he"

In [43]:
# 1. Load the txt file
with open('docs/kareem-01.txt', 'r', encoding='utf-8') as file:
    # Read the content of the file
    text = file.read()

text=text.replace("\\n", "\n").replace("\\", "");
print(text)

‫מבוא‬
‫מהי נאמנות‬

‫‪5‬‬

‫א‪.‬‬

‫‪ .1‬שליטה בנכם‬

‫‪7‬‬

‫‪ .2‬החובה לפעול לטובת זולתו‬

‫‪11‬‬

‫דרכים לנטרול הסיכונים הכרוכים בשליטה‬

‫‪17‬‬

‫‪ .1‬ריבוי נאמנים‬

‫‪17‬‬

‫‪ .2‬ישותנאמנותית‬

‫‪18‬‬

‫הדין בישראל לפני חוק הנאמנות‬

‫‪21‬‬

‫‪ .1‬החקיקה‬

‫‪21‬‬

‫‪ .2‬נאמנות בפסיקה‬

‫‪25‬‬

‫‪ .3‬התפישה הנזיקית‬

‫‪28‬‬

‫ד‪.‬‬

‫הצעת החוק ומקורותיה‬

‫‪33‬‬

‫ה‪.‬‬

‫הדיון בהצעת החוק‬

‫‪35‬‬

‫ו‪.‬‬

‫על כמה מעיקרי החוק‬

‫‪37‬‬

‫‪ .1‬הגדרת הנאמנות‬

‫‪37‬‬

‫‪ .2‬חובות הנאמן‬

‫‪41‬‬

‫״חובות אמוף‬

‫‪43‬‬

‫‪ .1‬מהן ״חובות אמוף‬

‫‪43‬‬

‫‪ .2‬חובת האמינות האוניברסלית‬

‫‪46‬‬

‫‪ .3‬חובת האמינות החוזית‬

‫‪49‬‬

‫‪ .4‬חובת האמינות המיגזרית‬

‫‪58‬‬

‫‪ .5‬חובת ההתחשבות‬

‫‪59‬‬

‫‪ .6‬חובת הנאמנות‬

‫‪63‬‬

‫‪ .7‬כוח חייבי האמון‬

‫‪67‬‬

‫‪ .8‬המערכות הנורמטיביות ‪ -‬השוואה‬

‫‪69‬‬

‫א‪ .‬החובה לפעול לטובת הזולת‬

‫‪69‬‬

‫ב‪.‬‬

‫ג‪.‬‬

‫ז‪.‬‬

‫‪3‬‬

‫ח‪.‬‬

‫ב‪ .‬הגנה על נכסי הזולת‬

‫‪70‬‬

‫ג‪ .‬ניגוד עניינים‬

‫‪71‬‬

‫ד‪ .‬מתן שירותים‬

‫‪72‬‬

‫ה‪ .‬חובת הסו

In [44]:
print("----- Cleaned Text -----")
print(clean_text(text))

----- Cleaned Text -----
‫מבוא‬
‫מהי נאמנות‬
‫‪5‬‬
‫א‪.‬‬
‫‪ .1‬שליטה בנכם‬
‫‪7‬‬
‫‪ .2‬החובה לפעול לטובת זולתו‬
‫‪11‬‬
‫דרכים לנטרול הסיכונים הכרוכים בשליטה‬
‫‪17‬‬
‫‪ .1‬ריבוי נאמנים‬
‫‪17‬‬
‫‪ .2‬ישותנאמנותית‬
‫‪18‬‬
‫הדין בישראל לפני חוק הנאמנות‬
‫‪21‬‬
‫‪ .1‬החקיקה‬
‫‪21‬‬
‫‪ .2‬נאמנות בפסיקה‬
‫‪25‬‬
‫‪ .3‬התפישה הנזיקית‬
‫‪28‬‬
‫ד‪.‬‬
‫הצעת החוק ומקורותיה‬
‫‪33‬‬
‫ה‪.‬‬
‫הדיון בהצעת החוק‬
‫‪35‬‬
‫ו‪.‬‬
‫על כמה מעיקרי החוק‬
‫‪37‬‬
‫‪ .1‬הגדרת הנאמנות‬
‫‪37‬‬
‫‪ .2‬חובות הנאמן‬
‫‪41‬‬
‫״חובות אמוף‬
‫‪43‬‬
‫‪ .1‬מהן ״חובות אמוף‬
‫‪43‬‬
‫‪ .2‬חובת האמינות האוניברסלית‬
‫‪46‬‬
‫‪ .3‬חובת האמינות החוזית‬
‫‪49‬‬
‫‪ .4‬חובת האמינות המיגזרית‬
‫‪58‬‬
‫‪ .5‬חובת ההתחשבות‬
‫‪59‬‬
‫‪ .6‬חובת הנאמנות‬
‫‪63‬‬
‫‪ .7‬כוח חייבי האמון‬
‫‪67‬‬
‫‪ .8‬המערכות הנורמטיביות ‪ -‬השוואה‬
‫‪69‬‬
‫א‪ .‬החובה לפעול לטובת הזולת‬
‫‪69‬‬
‫ב‪.‬‬
‫ג‪.‬‬
‫ז‪.‬‬
‫‪3‬‬
‫ח‪.‬‬
‫ב‪ .‬הגנה על נכסי הזולת‬
‫‪70‬‬
‫ג‪ .‬ניגוד עניינים‬
‫‪71‬‬
‫ד‪ .‬מתן שירותים‬
‫‪72‬‬
‫ה‪ .‬חובת הסודיות‬
‫‪73‬‬
‫ו‪ .‬תחולה בו‪-‬זמנית של חוב

In [45]:
# 2. Create Chunks

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200)
# Create chunks, extract text
texts = list(map((lambda x: x.page_content),
             text_splitter.create_documents([clean_text(text)])))
print_chunks(texts)

Total Chunks:  194
--------
CHUNK 1
--------
‫מבוא‬
‫מהי נאמנות‬
‫‪5‬‬
‫א‪.‬‬
‫‪ .1‬שליטה בנכם‬
‫‪7‬‬
‫‪ .2‬החובה לפעול לטובת זולתו‬
‫‪11‬‬
‫דרכים לנטרול הסיכונים הכרוכים בשליטה‬
‫‪17‬‬
‫‪ .1‬ריבוי נאמנים‬
‫‪17‬‬
‫‪ .2‬ישותנאמנותית‬
‫‪18‬‬
‫הדין בישראל לפני חוק הנאמנות‬
‫‪21‬‬
‫‪ .1‬החקיקה‬
‫‪21‬‬
‫‪ .2‬נאמנות בפסיקה‬
‫‪25‬‬
‫‪ .3‬התפישה הנזיקית‬
‫‪28‬‬
‫ד‪.‬‬
‫הצעת החוק ומקורותיה‬
‫‪33‬‬
‫ה‪.‬‬
‫הדיון בהצעת החוק‬
‫‪35‬‬
‫ו‪.‬‬
‫על כמה מעיקרי החוק‬
‫‪37‬‬
‫‪ .1‬הגדרת הנאמנות‬
‫‪37‬‬
‫‪ .2‬חובות הנאמן‬
‫‪41‬‬
‫״חובות אמוף‬
‫‪43‬‬
‫‪ .1‬מהן ״חובות אמוף‬
‫‪43‬‬
‫‪ .2‬חובת האמינות האוניברסלית‬
‫‪46‬‬
‫‪ .3‬חובת האמינות החוזית‬
‫‪49‬‬
‫‪ .4‬חובת האמינות המיגזרית‬
‫‪58‬‬
‫‪ .5‬חובת ההתחשבות‬
‫‪59‬‬
‫‪ .6‬חובת הנאמנות‬
‫‪63‬‬
‫‪ .7‬כוח חייבי האמון‬
‫‪67‬‬
‫‪ .8‬המערכות הנורמטיביות ‪ -‬השוואה‬
‫‪69‬‬
‫א‪ .‬החובה לפעול לטובת הזולת‬
‫‪69‬‬
‫ב‪.‬‬
‫ג‪.‬‬
‫ז‪.‬‬
‫‪3‬‬
‫ח‪.‬‬
‫ב‪ .‬הגנה על נכסי הזולת‬
‫‪70‬‬
‫ג‪ .‬ניגוד עניינים‬
‫‪71‬‬
‫ד‪ .‬מתן שירותים‬
‫‪72‬‬
‫ה‪ .‬חובת הסודיות‬
‫‪73‬‬
‫ו‪ .‬תחו

In [46]:
# 3. Reset Pinecone and Upsert Chunks. Only Run Once
# reset_pinecone(pinecone_namespace)
# embed_and_upsert(texts, pinecone_namespace)

In [47]:
from utils import get_relevant_chunks
from search_queries import kareem_01_queries

# https://chatgpt.com/share/271c8eea-0efc-4c6d-9e76-391a11eecc1b
queries = kareem_01_queries
query = queries[3]["hebrew"]
relevant_chunks = get_relevant_chunks(query, pinecone_namespace, 10)
relevant_chunks_text = [item["metadata"]["text"] for item in relevant_chunks]

print(pinecone_namespace)
print(query)
print_chunks(relevant_chunks, True)

simple-rag-1-he
מה הסיבות לאי תחולת חוק הנאמנות על סוגי נאמנויות מסוימות?
Total Chunks:  10
--------
CHUNK 1
--------
Id: 185, Score: 0.930361152
‫לעתים קרובות ללא גורם מקביל הרשאי לתת לו הוראות‪ ,‬ואף לשנות את מסלול‬
‫פעולתו‪ ,‬הטיל החוק תפקידים אלה על בתי‪-‬המשפט‪.‬‬
‫כך מוסמך בית־המשפט לתת לנאמן הוראות‪ ,‬החורגות מסוג ההוראות שבתי‪-‬‬
‫המשפט נוהגים לתת‪ .‬להוראות אלה עשוי להיות אופי מנהלי‪ ,‬עסקי ואף‬
‫קונםטיטוטיבי‪ ,‬לרבות מינוי נאמנים ופיטוריהם‪ ,‬הנחייתם בבעיות השונות שהם‬
‫עשויים להיתקל בהן‪ ,‬ובמקרים מסוימים ‪ -‬גם סמכות לשנות זכויותיהם של נהנים‪.‬‬
‫בית‪-‬המשפט ממלא תפקידים אלה במקרים שעל‪-‬פי תנאי הנאמנות ונסיבותיה אין‬
‫גורם אחר המוסמך לקבל החלטות מעין אלה‪ .‬בפועל אמורים הדברים בנאמנות‬
‫על־פי כתב־הקדש‪ .‬לגבי נאמנות על‪-‬פי חוק יש בחוק המיוחד לכל סוג נאמנים‬
‫מנגנון המסדיר נושאים אלה במידה מסוימת‪ .‬לגבי נאמנות על‪-‬פי חוזה חלים דיני‪-‬‬
‫החוזים‪ ,‬והם משאירים נושאים אלה להחלטותיהם המשותפות של הצדדים לחוזה‪.‬‬
‫יא‪.‬‬
‫רשם ההקדשות‬
‫§‪ 27‬נאמנויות לצורכי ציבור עשויות להיות

In [48]:
from utils import rerank_chunks

# Lets use a reranker to see how much it helps
rerank_results = rerank_chunks(query, relevant_chunks_text)
print_chunks(rerank_results, True)

Total Chunks:  10
--------
CHUNK 1
--------
Id: 4, Score: 0.73046875
‫מיוחד על־פני חוק הנאמנות‪ .‬עם זאת‪ ,‬לגבי כמה סוגי נאמנויות נבחנה מידת‬
‫התחולה של הוראות חוק הנאמנות לגביהם; שכן בהיותו חוק משלים‪ ,‬חלות לעתים‬
‫הוראותיו גם על נאמנויות שלכאורה הוסדרו בחוקים מיוחדים‪.‬‬
‫בנושאי ההגנה בפני הנאמן שאב המחוקק השראה מהמשפט האנגלו־‬
‫אמריקני‪ .‬לכן‪ ,‬למרות השוני בהנמקה העיונית‪ ,‬הובאו בהערות פסקי‪-‬דין וכן‬
‫מראי־מקומות לחיבורים מהספרות המשפטית האנגלית והאמריקנית לשם‬
‫השוואת הפתרונות לבעיות מעשיות‪ .‬לעניין זה הניסיון האנגלו־אמריקני מאלף‬
‫ולכן מועיל‪.‬‬
‫בפסיקה ניתן למצוא ראשיתה של התייחסות לחוק‪ ,‬במיוחד בהקשר עם‬
‫צוואות ונאמנות על‪-‬פי חוזה‪ .‬בפועל גם הפסיקה שלאחר תחולת החוק מתייחסת‬
‫עדיין ברובה לדין ששרר לפני מועד תחולתו‪.‬‬
‫פסיקת בתי‪-‬המשפט המתייחסת לדין שקדם לחוק הנאמנות‪ ,‬הובאה על יסוד‬
‫ההנחה שגם בעתיד יונחו בתי‪-‬משפט על‪-‬ידי פסקי־הדין שניתנו על־ידם בעבר‪,‬‬
‫להארת עקרונותיו של חוק הנאמנות‪.‬‬
‫סעיף ‪ 39‬לחוק הנאמנות‪.‬‬
‫‪90‬‬
--------
CHUNK 2
--------
Id: 9, Score: 

In [49]:
# Only the Top Chunks should be given to llm, so we choose chunks with a score of 0.8
top_chunks = [item for item in rerank_results if item["score"] >= 0.6]
print_chunks(top_chunks, True)

Total Chunks:  2
--------
CHUNK 1
--------
Id: 4, Score: 0.73046875
‫מיוחד על־פני חוק הנאמנות‪ .‬עם זאת‪ ,‬לגבי כמה סוגי נאמנויות נבחנה מידת‬
‫התחולה של הוראות חוק הנאמנות לגביהם; שכן בהיותו חוק משלים‪ ,‬חלות לעתים‬
‫הוראותיו גם על נאמנויות שלכאורה הוסדרו בחוקים מיוחדים‪.‬‬
‫בנושאי ההגנה בפני הנאמן שאב המחוקק השראה מהמשפט האנגלו־‬
‫אמריקני‪ .‬לכן‪ ,‬למרות השוני בהנמקה העיונית‪ ,‬הובאו בהערות פסקי‪-‬דין וכן‬
‫מראי־מקומות לחיבורים מהספרות המשפטית האנגלית והאמריקנית לשם‬
‫השוואת הפתרונות לבעיות מעשיות‪ .‬לעניין זה הניסיון האנגלו־אמריקני מאלף‬
‫ולכן מועיל‪.‬‬
‫בפסיקה ניתן למצוא ראשיתה של התייחסות לחוק‪ ,‬במיוחד בהקשר עם‬
‫צוואות ונאמנות על‪-‬פי חוזה‪ .‬בפועל גם הפסיקה שלאחר תחולת החוק מתייחסת‬
‫עדיין ברובה לדין ששרר לפני מועד תחולתו‪.‬‬
‫פסיקת בתי‪-‬המשפט המתייחסת לדין שקדם לחוק הנאמנות‪ ,‬הובאה על יסוד‬
‫ההנחה שגם בעתיד יונחו בתי‪-‬משפט על‪-‬ידי פסקי־הדין שניתנו על־ידם בעבר‪,‬‬
‫להארת עקרונותיו של חוק הנאמנות‪.‬‬
‫סעיף ‪ 39‬לחוק הנאמנות‪.‬‬
‫‪90‬‬
--------
CHUNK 2
--------
Id: 9, Score: 0